In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from uuid import uuid4
from tqdm.auto import tqdm
import openai 
import pinecone
import xml.etree.ElementTree as ET
import re
import html
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
import datetime
from time import sleep
import os
# from _collections_abc import MutableMapping
# import collections
# collections.Callable = collections.abc.Callable
from IPython.display import Markdown

openai.api_key = os.getenv("OPENAI_API_KEY") 
pinecone_api_key = os.getenv("PINECONE_API_KEY")


In [4]:

url = 'https://apidocs.sugarcrm.com/schema/10.0.6/ent/sugar.sugar.xml'
response = requests.get(url)
xml_content = response.content

root = ET.fromstring(xml_content)

results = []

# Iterate through table elements
for table in root.findall('.//table'):
    table_name = table.get('name')

    # Iterate through primaryKey elements and add them to the results
    for primary_key in table.findall('.//primaryKey'):
        primary_key_column = primary_key.get('column')
        results.append({'table': table_name, 'column': primary_key_column, 'key': 'Primary Key', 'linked_table': ''})

    # Iterate through column elements
    for column in table.findall('.//column'):
        column_name = column.get('name')
        foreign_key = ''
        other_table = ''

        # Iterate through child elements
        child = column.find('.//child')
        if child is not None:
            foreign_key = 'Foreign Key'
            other_table = child.get('table')

        # Iterate through parent elements
        parent = column.find('.//parent')
        if parent is not None:
            foreign_key = 'Foreign Key'
            other_table = parent.get('table')

        # Add the column to the results only if it's not a primary key
        if foreign_key != 'Primary Key':
            results.append({'table': table_name, 'column': column_name, 'key': foreign_key, 'linked_table': other_table})

# Remove duplicates from results
unique_results = []
for result in results:
    if result not in unique_results:
        unique_results.append(result)



In [8]:
text_list = unique_results

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [12]:


# Convert the list of dictionaries into a list of strings prior to tokenizing the text

def convert_to_text_list(unique_results: List[dict]) -> List[str]:
    text_list = []
    for entry in unique_results:
        text = f"table:{entry['table']}, column:{entry['column']}, key:{entry['key']}, linked_table:{entry['linked_table']}"
        text_list.append(text)
    return text_list


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,  
    separators=["\n"]
)

text_list = convert_to_text_list(unique_results)  

text_string = "\n".join(text_list)

chunks = text_splitter.split_text(text_string)

In [13]:

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)


In [14]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = 'sugar-schema-chatbot-ver'


pinecone.init(
    pinecone_api_key=pinecone_api_key 
    environment="us-east-1-aws"  
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

index = pinecone.GRPCIndex(index_name)


In [16]:

texts = chunks

batch_size = 200  

for i in tqdm(range(0, len(texts), batch_size)):
    # find end of batch
    i_end = min(len(texts), i + batch_size)
    texts_batch = texts[i:i_end]

    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts_batch, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts_batch, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]

    # Create a list of tuples with index (as a string), embeddings, and metadata (text)
    to_upsert = [(str(idx), embed, {"text": text}) for idx, embed, text in zip(range(i, i_end), embeds, texts_batch)]


    index.upsert(vectors=to_upsert)

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:

query = "drawing on the documentation write me a sql query that joins the accounts, accounts_cstm, user and teams tables, \
only use foreign and primary keys that are linked between tables in the schema"


res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=10, include_metadata=True)

contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [21]:

# system message to 'prime' the model

primer = f"""You are Q&A bot with skills in SQL. You are able to answer any question about the SugarCRM database schema.
Follow instructions below to answer the user's question.
"""

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)
     

In [22]:
""" Display the answer """""


display(Markdown(res['choices'][0]['message']['content']))

Based on the provided schema and the linked tables you mentioned, you can use the following SQL query to join the accounts, accounts_cstm, users, and teams tables:

```sql
SELECT a.*, ac.*, u.*, t.*
FROM accounts a
JOIN accounts_cstm ac ON a.id = ac.id_c
JOIN users u ON a.assigned_user_id = u.id
JOIN teams t ON a.team_id = t.id;
```

This query selects all columns from the accounts table (`a.*`), accounts_cstm table (`ac.*`), users table (`u.*`), and teams table (`t.*`). Then it joins these tables using the Foreign Key relations present in the schema:

- Join between `accounts` and `accounts_cstm` tables: `a.id = ac.id_c`
- Join between `accounts` and `users` tables: `a.assigned_user_id = u.id`
- Join between `accounts` and `teams` tables: `a.team_id = t.id`